In [ ]:
%matplotlib widget

import os
import numpy as np
import matplotlib.pyplot as plt

def load_file(filename):
    """
    Reads complex samples from the rtlsdr file.

    Parameters:
    filename (str): The name of the file to read from.

    Returns:
    np.ndarray: The complex samples read from the file.
    """
    with open(filename, 'rb') as fid:
        y = np.fromfile(fid, dtype=np.uint8).astype(np.double)

    y = y - 127
    y = y[0::2] + 1j * y[1::2]

    return y


y_absd_3_2_3 = np.abs(load_file("adsb_3.2M_3.dat"))
y_absd_3_2 = np.abs(load_file("adsb_3.2M_3.dat"))
y_absd_32 = np.abs(load_file("adsb_3.2M_3.dat"))

# Size of all examples is the same, can have same time scale
fd_hz = 3.2e6
t_s = np.arange(0, len(y_absd_3_2_3)) / fd_hz
t_us = t_s * 1e6

plt.figure(figsize=(10, 10))
plt.suptitle("Full sample ADS-B signals")
plt.subplot(311)
plt.title("adsb_3.2M_3.dat data")
plt.plot(t_us, y_absd_3_2_3)
plt.xlabel("t, us")
plt.ylabel("Amplitude")
plt.subplot(312)
plt.title("adsb_3.2M_3.dat data")
plt.plot(t_us, y_absd_3_2)
plt.xlabel("t, us")
plt.ylabel("Amplitude")
plt.subplot(313)
plt.title("adsb_3.2M_3.dat data")
plt.plot(t_us, y_absd_32)
plt.xlabel("t, us")
plt.ylabel("Amplitude")
plt.tight_layout()
plt.show()

In [ ]:
adsb_signal_length_samples = int((112 + 8) * (fd_hz / 1e6))
adsb_signal_length_samples_for_capture = adsb_signal_length_samples + 10
random_signal_amount = 10

# Extracts random 112 + 8us samples above threshold
def extract_random_signal(signal: np.ndarray, threshold: int=20) -> np.ndarray:
    while(True):
        random_index = np.random.randint(0, len(signal))
        for i in range(random_index, len(signal) - random_index - adsb_signal_length_samples_for_capture):
            if signal[i] < threshold:
                continue
            signal_samples = signal[(i - 5):(i + adsb_signal_length_samples_for_capture + 5)]
            if np.mean(signal_samples) < 3:
                continue
            return signal_samples

def extract_random_signals(signal : np.ndarray, label: str) -> np.ndarray:
    random_adsb_signals = [extract_random_signal(signal) for i in range(random_signal_amount)]
    plt.figure(figsize=(10, 13))
    plt.suptitle(label)
    for i in range(len(random_adsb_signals)):
        plt.subplot(len(random_adsb_signals), 1, i + 1)
        plt.plot(random_adsb_signals[i])
        plt.ylabel("Amplitude")
    plt.xlabel("Samples")
    plt.tight_layout()
    plt.show()
    return random_adsb_signals

random_y_absd_3_2_3 = extract_random_signals(y_absd_3_2_3, "adsb_3.2M_3.dat data")
randomy_absd_3_2 = extract_random_signals(y_absd_3_2, "adsb_3.2M_3.dat data")
random_y_absd_32 = extract_random_signals(y_absd_32, "adsb_3.2M_3.dat data")
